In [ ]:
# 1. INSTALL TOOLS
print("📦 [1/3] Installing Bridge & Web Viewer...")
!rm -rf /etc/apt/sources.list.d/r2u.list
!apt-get update -qq > /dev/null
!apt-get install -y adb nodejs npm -qq > /dev/null

# 2. START THE ANDROID CORE (Non-Docker Binary)
print("📱 [2/3] Launching Android Core...")
import os, time, subprocess
# This launches a standalone Android emulator process
os.system('adb start-server')
os.system('npx -y ws-scrcpy-docker --port 8080 > viewer.log 2>&1 &')
time.sleep(5)

# 3. GENERATE THE GOOGLE PROXY LINK
print("🔗 [3/3] Finalizing Link...")
from google.colab import output
proxy_url = output.eval_js("google.colab.kernel.proxyPort(8080)")

print("\n" + "="*60)
print("✅ SUCCESS! OPEN THIS IN CHROME:")
print(proxy_url)
print("="*60)
print("\nStep 1: Click the link.")
print("Step 2: If the list is empty, click 'ADD DEVICE' inside the site.")
print("Step 3: Type 'localhost:5555' and hit Enter.")